In [ ]:
using Evolutionary
using Flux
using Flux: onehot, onecold, logitcrossentropy, onehotbatch, crossentropy
using MLDatasets
using Random
using Statistics
using MLJBase
using Printf
using BSON: @load # for load weights
using Plots
using DelimitedFiles
import Evolutionary.initial_population
using Zygote
import Evolutionary.NonDifferentiable
import Base: copy, copyto!

In [ ]:
Random.seed!(63456345)

In [14]:
rawdata = readdlm("data/test2.csv",',',skipstart=1)';
one = 0
zero = 0
for i in rawdata[20,:]
    if i == 1
        one = one + 1
    else
        zero = zero + 1
    end
end

difference = one - zero

function newrand()
    randCol = zeros(0)
    push!(randCol, rand(10.0:99.0))
    push!(randCol, rand(5.3:40.2))
    push!(randCol, rand(1.1:15.1))
    push!(randCol, rand(0.1:8.1))
    push!(randCol, rand(1.1:15.1))
    push!(randCol, rand(30.1:60.1))
    push!(randCol, rand(0.1:0.9))
    push!(randCol, rand(0.1:4.1))
    push!(randCol, rand(20.1:50.1))
    push!(randCol, rand(0.1:5.1))
    push!(randCol, rand(0.1:7.1))
    push!(randCol, rand(45.1:99.1))
    push!(randCol, rand(0.1:5.1))
    push!(randCol, rand(0.1:5.1))
    push!(randCol, rand(0.1:7.1))
    push!(randCol, rand(0.1:7.1))
    push!(randCol, rand(0.1:3.1))
    push!(randCol, rand(0.1:3.1))
    push!(randCol, rand(0.1:5.1))
    if rand(0:1) == 0
        push!(randCol, 0.0)
    else
        push!(randCol, 1.0)
    end
    return randCol
end

for i = 1:size(rawdata,2)
    if rawdata[20,i] == 0 && difference > 0
        rawdata = hcat(rawdata, rawdata[:,i])
        rawdata = hcat(rawdata, newrand())

        difference = difference - 1
    end
end
filldata = rawdata[ :, shuffle(1:end)];

x = filldata[1:19, :]
y = filldata[20, :];

x_train = x[:,1:floor(Int, size(x,2)*0.7)]
y_train = y[1:floor(Int, size(x,2)*0.7)]
x_test = x[:,floor(Int, size(x,2)*0.7)+1:end]
y_test = y[floor(Int, size(x,2)*0.7)+1:end];


In [3]:
train_data = [ (x, onehot(l, unique(y_train))) for (x, l) in zip(eachcol(x_train), y_train)]
test_data = [ (x, onehot(l, unique(y_test))) for (x, l) in zip(eachcol(x_test), y_test)];

In [18]:
accuracy(xy, model) = mean( onecold(model(x)) .== onecold(y) for (x,y) in xy)
loss(model) = (x,y)->crossentropy(model(x), y)
loss(model,x,y) = loss(model)(x, y)
loss(xy, model) = loss(model)(hcat(map(first,xy)...), hcat(map(last,xy)...))

loss (generic function with 3 methods)

In [19]:
fitness(m) = loss(train_data, m)

fitness (generic function with 1 method)

In [6]:
function uniform_mlp(m1::T, m2::T; rng::Random.AbstractRNG=Random.default_rng()) where {T <: Chain}
    θ1, re1 = Flux.destructure(m1);
    θ2, re2 = Flux.destructure(m2);
    c1, c2 = UX(θ1,θ2; rng=rng)
    return re1(c1), re2(c2)
end

function gaussian_mlp(σ::Real = 1.0)
    vop = gaussian(σ)
    function mutation(recombinant::T; rng::Random.AbstractRNG=Random.default_rng()) where {T <: Chain}  
        θ, re = Flux.destructure(recombinant)
        return re(convert(Vector{Float32}, vop(θ; rng=rng)))
    end
    return mutation
end

gaussian_mlp (generic function with 2 methods)

In [7]:
function initial_population(method::M, individual::Chain;
    rng::Random.AbstractRNG=Random.default_rng(),
    kwargs...) where {M<:Evolutionary.AbstractOptimizer}
θ, re = Flux.destructure(individual);
[re(randn(rng, length(θ))) for i in 1:Evolutionary.population_size(method)]
end

initial_population (generic function with 6 methods)

In [20]:
NonDifferentiable(f, x::Chain) = NonDifferentiable{Real,typeof(x)}(f, f(x), deepcopy(x),[0,])
copy(ch::Chain) = deepcopy(ch)
# copy weight and bias between two models
function copyto!(layer1::Dense{T}, layer2::Dense{T}) where {T}
    copyto!(layer1.W, layer2.W)
    copyto!(layer1.b, layer2.b)
    return l1
end
function copyto!(ch1::Chain, ch2::Chain)
    for i in 1:length(ch1.layers)
        copyto!(ch1.layers[i],ch2.layers[i])
    end
    return ch1  
end

copyto! (generic function with 2 methods)

In [9]:
# opts = Evolutionary.Options(iterations=100, successive_f_tol=25, rng=StableRNG(42))
# algo = GA(
#         selection = rouletteinv,
#         mutation =  gaussian_mlp(),
#         crossover = uniform_mlp,
#         mutationRate = 0.2,
#         crossoverRate = 0.9,
#         populationSize = 1600,
#         ε = 0.0003
#     )

UndefVarError: UndefVarError: StableRNG not defined

In [21]:
opts = Evolutionary.Options(iterations=100)
algo = ES(
    selection = :comma,
    mu = 10,
    lambda = 70
)

ES(NoStrategy(), first, first, Evolutionary.var"#42#45"(), identity, 10, 1, 70, :comma)

In [24]:
model = Chain(Dense(19, 2*19, sigmoid), Dense(2*19, 2))

Chain(
  Dense(19, 38, σ),                     # 760 parameters
  Dense(38, 2),                         # 78 parameters
)                   # Total: 4 arrays, 838 parameters, 3.523 KiB.

In [26]:
res = Evolutionary.optimize(fitness, model, algo, opts)

DomainError: DomainError with -3.938436729305889:
log will only return a complex result if called with a complex argument. Try log(Complex(x)).